In [159]:
import os
os.chdir("/Users/billydodds/Documents/Projects/Transactions_ML")

import pandas as pd
import numpy as np
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import LinearSVC
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
# from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
# from sklearn.neural_network import MLPClassifier

from components.scripts.load_data import load_data
from components.scripts.MyDecisionTree import MyDecisionTree
from components.scripts.process_data import get_corpora, NLP_distances, run_fold

import multiprocess as multiprocessing # normal multiprocessing doesn't work in Jupyter for some reason

from gensim.models import Word2Vec

# import random

# import itertools

In [160]:
tr_data = load_data("CSVData.csv")
y = tr_data[~pd.isnull(tr_data.category)].category
X = tr_data[~pd.isnull(tr_data.category)].drop("category", axis=1)

X = X.drop(["description", "desc_corpus"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

X_train["category"] = y_train
# corpora = get_corpora(X_train)
# X_train = X_train.drop("category", axis=1)

# X_train = NLP_distances(X_train, corpora)
# X_test = NLP_distances(X_test, corpora)


X_train

<ipython-input-160-02329dc7a0c7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["category"] = y_train


,amount,date,weekday,desc_features,category
560,-6.20,2019-09-25,2,mcdonalds stanmore,food
238,-9.00,2020-02-10,0,apex petroleum west ryde,transport
557,-13.00,2019-09-24,1,alfred hotel mngmnt camperdown,beers
197,-37.99,2020-02-27,3,eleven rozelle,transport
286,-11.21,2020-02-02,6,aldi stores gladesvi gladesville,shopping
...,...,...,...,...,...
71,-7.40,2020-06-20,5,haps hotels gladesville,beers
489,-4.20,2019-10-28,0,taste baguette universit,food
237,-2.50,2020-02-13,3,sfs sydney,food
519,-9.95,2019-10-18,4,mcdonalds stanmore,food


In [161]:
sentences = []
for desc, cat in zip(X_train.desc_features, X_train.category):
    sentence = desc.split(" ")
#     categories = [cat for i in range(len(sentence))]
    splice = []
    for word in sentence:
        splice.append(cat)
        splice.append(word)
    splice.append(cat)
    
    sentences.append(splice)
    
print(sentences[6])

print("Number of sentences: {}.".format(len(voc)))
print("Number of rows: {}.".format(len(X_train)))

['food', 'gumption', 'food', 'coffee', 'food', 'sydney', 'food']
Number of sentences: 450.
Number of rows: 450.


In [162]:
w2v_model = Word2Vec(min_count=0,
                     window=1,
                     size=150,
#                      sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=0,
                     workers=16-1)

In [163]:
w2v_model.build_vocab(sentences, progress_per=10000)

In [164]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(45023, 110940)

In [165]:
w2v_model.init_sims(replace=True)


In [166]:
w2v_model.wv.similarity('eleven', 'food')

0.031985078

In [167]:
w2v_model.wv.similarity('life/wellbeing', 'transport')

0.038791016

In [169]:

def get_similarity(words, cat):
    sims = []
    for word in words.split(" "):
        try:
            sims.append(w2v_model.wv.similarity(word, cat))
        except KeyError:
            pass
    if len(sims) == 0:
        return -1.0
    return np.mean(sims)

weights = {"amount":5, "weekday":5}

for cat in X_train.category.unique():
    weights[cat + "_desc_dist"] = 1
#     X_train["word2vec_dist_"+cat] = [get_similarity(desc, cat) for desc in X_train.desc_features]
#     X_test["word2vec_dist_"+cat] = [get_similarity(desc, cat) for desc in X_test.desc_features]

weights

{'amount': 5,
 'weekday': 5,
 'food_desc_dist': 1,
 'transport_desc_dist': 1,
 'beers_desc_dist': 1,
 'shopping_desc_dist': 1,
 'wages_desc_dist': 1,
 'life/wellbeing_desc_dist': 1,
 'transfer_desc_dist': 1}

In [155]:
X_train = X_train.drop(["desc_features", "category", "date"], axis=1)
X_test = X_test.drop(["desc_features", "date"], axis=1)



In [157]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
np.mean(predictions == y_test)

0.78

In [ ]:
def run_fold_word2vec(X:pd.DataFrame, split:Tuple[np.array, np.array], model:Any, web_scrape=False, lookup=False, verbose=True, min_freq=0) -> float:
    train_index, test_index = split
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = X_train.category, X_test.category

    # Web Scraping:
    if web_scrape:
        goog = [google(query, verbose) for query in X_test.desc_features]
        web_predictions = np.array([categorise(goo, verbose) for goo in goog], dtype=object)
    
    if lookup:
        table = pd.DataFrame(X_train[["desc_features", "category"]].groupby(["desc_features"])["category"].unique().apply(','.join))
        table = table.reset_index()
        table = table[~table.category.str.contains(",")]
        lookup_predictions = np.array(X_test.drop("category", axis=1).merge(table, on="desc_features", how="left", validate="many_to_one").category, dtype=object)

    corpus = get_corpora(X_train, min_freq=min_freq)
    X_train = NLP_distances(X_train.drop('category', axis=1), corpus)
    X_test = NLP_distances(X_test.drop('category', axis=1), corpus)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scale(X_train, scaler)
    X_test = scale(X_test, scaler)
    

    model.fit(X_train, y_train)
    model_predictions = model.predict(X_test)
    predictions = model_predictions
    

    if web_scrape or lookup:
        print("model: ", predictions, type(predictions)) if verbose else None
        if web_scrape:
            print("web: ", web_predictions, type(web_predictions)) if verbose else None
            predictions = np.array([web if web != "" else mod for web, mod in zip(web_predictions, predictions)])
        if lookup:
            print("lookup: ", lookup_predictions, type(lookup_predictions)) if verbose else None
            predictions = np.array([look if str(look) != 'nan' else pred for look, pred in zip(lookup_predictions, predictions)])
        corrections = np.array([1 if pred!=mod else 0 for pred, mod in zip(predictions, model_predictions)])

        baseline = np.array([1 if str(look) != 'nan' else 0 for look in lookup_predictions])

    print("predictions: ", predictions) if verbose else None
    print("actual: ", np.array(y_test)) if verbose else None
    print("matches: ", predictions == np.array(y_test)) if verbose else None

    acc = sum(predictions == y_test)/len(predictions)

    if verbose and (web_scrape or lookup):
        print(f"corrections: {sum(corrections)} of {len(corrections)}")
        print(f"Baseline (just lookup without ML model): {sum(baseline)/len(baseline)}")
    
    print(acc) if verbose else None
    return acc